## Идеальное решение на основе предыдущих экспериментов

Из ноутбука удалены лишние строки по анализу данных и прошлых экспериментов
чтобы уменьшить время выполнения ноутбука да и вообще самому легче ориентироваться в коротком ноутбуке

Использовать **ImageDataAugmentor** не получилось, постоянно возникали какие-то непонятные ошибки, решил настраивать генерацию руками

In [ ]:
#!pip install tensorflow --upgrade
!pip install -q efficientnet
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor

In [ ]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import csv
import os
import sys
import zipfile
import shutil

import tensorflow as tf
import efficientnet.tfkeras as efn

import keras as keras
import keras.models
import keras.layers
import keras.backend
import keras.callbacks

from ImageDataAugmentor.image_data_augmentor import *
import albumentations

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

from keras import optimizers
from keras.models import Model, Sequential
from keras.callbacks import Callback, LearningRateScheduler, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from keras.layers import *
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, KFold

import PIL
from PIL import ImageOps, ImageFilter
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

In [ ]:
!pip freeze > requirements.txt

In [ ]:
!nvidia-smi

In [ ]:
device_list = device_lib.list_local_devices()
device_list_GPU = [x.name for x in device_list if 'GPU' in x.name]
print ('GPU подключен') if device_list_GPU else  print('GPU не подключен')

# Setup

In [ ]:
INPUT_PATH  = '../input/sf-dl-car-classification/'
PICTURE_PATH = '/dev/shm/'
PICTURE_SPLIT_PATH = '/dev/shm/split/'
MODELS_PATH = '../input/sf-dl-car-classification/modelas12/'
OUTPUT_PATH = '/kaggle/working/car/'

In [ ]:
os.makedirs(PICTURE_PATH, exist_ok = True)
os.makedirs(PICTURE_SPLIT_PATH, exist_ok = True)


RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)

In [ ]:
EPOCHS       = 7
BATCH_SIZE   = 32
LR           = 1e-3
END_LR       = 1e-4
DECAY_STEPS  = 100000
VALID_SPLIT  = 0.2

CLASS_NUM    = 10
IMG_SIZE     = 280
IMG_CHANNELS = 3
input_shape  = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

USE_BIAS     = False
KERNEL_REG   = 'l2'
DROPOUT_RATE = 0.25
EPOCHS_DROP  = 1

# EDA / Анализ данных

In [ ]:
!rm -d -r '/dev/shm/'
#!rm -d -r '../working/car/'

In [ ]:
#!unzip '../input/sf-dl-car-classification/train.zip' -d /dev/shm/
#!unzip '../input/sf-dl-car-classification/train.zip' -d '../working/car/'
print('Распаковка картинок')
with zipfile.ZipFile(INPUT_PATH + 'train.zip',"r") as z:
    z.extractall(PICTURE_PATH)
print('Распаковка завершена')    

In [ ]:
train_df = pd.read_csv(INPUT_PATH + 'train.csv')

In [ ]:
categories = train_df.Category.value_counts()
categories

# Data

### Stratify Split
применить KFold не увенчалось успехом, так как постоянно приходилось перетасовывать изображения, что накладно, да и 3 этапа сами по себе долгие, если ещё и делить на разные комбинации, то вообще обучать будет вечность.

In [ ]:
def stratify(): 
    train_files, valid_files, train_labels, valid_labels = \
        train_test_split(train_df['Id'], train_df['Category'], 
                         test_size = VALID_SPLIT, 
                         random_state = RANDOM_SEED, 
                         stratify = train_df['Category'])

    train_files = pd.DataFrame(train_files)
    valid_files = pd.DataFrame(valid_files)
    train_files['Category'] = train_labels
    valid_files['Category'] = valid_labels

    print(train_files.shape, valid_files.shape)
    return train_files, valid_files

train_files, valid_files = stratify()

In [ ]:
def move_files():
    for cat in categories.index:
        os.makedirs(f'{PICTURE_SPLIT_PATH}train/{str(cat)}')
        os.makedirs(f'{PICTURE_SPLIT_PATH}valid/{str(cat)}') 
        
    count_file = 0
    for index, row in train_files.iterrows():
        file_path = 'train/' + str(row['Category']) + '/' + str(row['Id'])
        shutil.move(PICTURE_PATH + file_path, PICTURE_SPLIT_PATH + file_path)
        count_file += 1
    print(f'move {count_file} train files')
    
    count_file = 0
    for index,row in valid_files.iterrows():
        source_path = 'train/' + str(row['Category']) + '/' + str(row['Id'])
        destination_path = 'valid/' + str(row['Category']) + '/' + str(row['Id'])
        shutil.move(PICTURE_PATH + source_path, PICTURE_SPLIT_PATH + destination_path)
        count_file += 1
    print(f'move {count_file} valid files')  
    
move_files()

In [ ]:
total = 0
for i in range(0, 10):
    dirr = PICTURE_SPLIT_PATH + f'train/{i}/'
    count = len([name for name in os.listdir(dirr) if os.path.isfile(os.path.join(dirr, name))])
    print(f'{i} - ', count)
    total += count
print(f'total is train {total}')

total = 0
for i in range(0, 10):
    dirr = PICTURE_SPLIT_PATH + f'valid/{i}/'
    count = len([name for name in os.listdir(dirr) if os.path.isfile(os.path.join(dirr, name))])
    print(f'{i} - ', count)
    total += count
print(f'total is valid {total}')

### Data augmentation

In [ ]:
# для размера 380 и EfficientNetB4 получаем на последнем этапе ошибку 'OOM when allocating tensor ... by allocator GPU_0_bfc'

In [ ]:
p_rescale = 1. / 255
p_rotation_range = 5
p_zoom_range = 0.1
p_width_shift_range = 0.1
p_height_shift_range = 0.1
p_brightness_range = [0.5, 0.1]
p_shear_range = 0.15

In [ ]:
def create_dataGenerators(t):    
    if (t == 1):        
        datagen = ImageDataGenerator(
            rescale = p_rescale,
            zoom_range = p_zoom_range,
            rotation_range = p_rotation_range,
            width_shift_range = p_width_shift_range,
            height_shift_range = p_height_shift_range,
            shear_range = p_shear_range,
            horizontal_flip = True)
    else:
        AUGMENTATIONS = albumentations.Compose([
            albumentations.HorizontalFlip(p = 0.5),
            albumentations.Rotate(limit = 5, interpolation = 1, border_mode = 4, value = None, mask_value = None, always_apply = False, p = 0.5),
            albumentations.OneOf([
                albumentations.CenterCrop(height = 280, width = 260),
                albumentations.CenterCrop(height = 260, width = 280),
            ],p = 0.5),
            albumentations.OneOf([
                albumentations.RandomBrightnessContrast(brightness_limit = 0.3, contrast_limit = 0.3),
                albumentations.RandomBrightnessContrast(brightness_limit = 0.1, contrast_limit = 0.1)
            ],p = 0.5),
            albumentations.GaussianBlur(p = 0.05),
            albumentations.HueSaturationValue(p = 0.5),
            albumentations.RGBShift(p = 0.5),
            albumentations.FancyPCA(alpha = 0.1, always_apply = False, p = 0.5),
            albumentations.Resize(IMG_SIZE, IMG_SIZE)
        ])        
        datagen = ImageDataAugmentor(rescale = p_rescale, augment = AUGMENTATIONS)
        
    return datagen

datagen = create_dataGenerators(2)

### datagen

In [ ]:
def create_generators():
    train_generator = datagen.flow_from_directory(
        PICTURE_SPLIT_PATH + 'train/',
        target_size = (IMG_SIZE, IMG_SIZE),
        batch_size = BATCH_SIZE,
        class_mode = 'categorical',
        shuffle = True, 
        seed = RANDOM_SEED)

    valid_generator = datagen.flow_from_directory(
        PICTURE_SPLIT_PATH +'valid/',
        target_size = (IMG_SIZE, IMG_SIZE),
        batch_size = BATCH_SIZE,
        class_mode = 'categorical',
        shuffle = True, 
        seed = RANDOM_SEED)
    return train_generator, valid_generator

train_generator, valid_generator = create_generators()

# Строим модель

In [ ]:
# Для размера 260 используем EfficientNetB2
base_model = efn.EfficientNetB7(weights = 'imagenet', include_top = False, input_shape = input_shape)

In [ ]:
# Замораживаем веса в базовой модели
base_model.trainable = False

In [ ]:
def create_model():
    # Устанавливаем новую "голову" (head)
    model = Sequential()
    model.add(base_model)
    
    model.add(GlobalAveragePooling2D()) # объединяем все признаки в единый вектор     
    model.add(BatchNormalization())
    
    model.add(Dense(256, use_bias = USE_BIAS, kernel_regularizer = KERNEL_REG, activation = 'relu'))
    #model.add(BatchNormalization())
    
    model.add(Dropout(DROPOUT_RATE))
    model.add(Dense(CLASS_NUM, activation = 'softmax'))    
    
    model.summary()
    
    return model

def create_callbacks():
    checkpoint = ModelCheckpoint('best_model.hdf5', monitor = 'val_accuracy', verbose = 1, mode = 'max', save_best_only = True)
    earlystop = EarlyStopping(monitor = 'val_loss', min_delta = 0, verbose = 1, patience = 3, restore_best_weights = True)    
    def step_decay(epoch):
        return LR * math.pow(0.9, math.floor((1 + epoch) / EPOCHS_DROP))
    lrScheduler = LearningRateScheduler(step_decay, verbose = 1)
    #reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.25, patience = 3, min_lr = 0.0000001, verbose = 1, mode = 'auto')
    
    #tbCallBack = keras.callbacks.TensorBoard(log_dir = OUTPUT_PATH + 'logs/', histogram_freq = 0, write_graph = True, write_images = False)
    
    return [checkpoint, earlystop, lrScheduler]

callbacks_list = create_callbacks()

def build_and_fit_model(need_load = False, step_number = ''):    
    model = create_model()    
    model.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(lr = LR, amsgrad = True), metrics = ["accuracy"])       
    #model.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0), metrics = ["accuracy"])
    #model.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(learning_rate = LR, rho = 0.9, epsilon = 1e-08, decay = 0.0), metrics = ["accuracy"])       
    train_generator, valid_generator = create_generators()
    if need_load:
        history = None
        model.load_weights(MODELS_PATH + f'model_step_{step_number}.hdf5')
    else:        
        history = model.fit_generator(
            train_generator,
            steps_per_epoch = train_generator.samples//train_generator.batch_size,
            validation_data = valid_generator, 
            validation_steps = valid_generator.samples//valid_generator.batch_size,
            epochs = EPOCHS,
            callbacks = callbacks_list
        )
    return history, model

In [ ]:
history, model = build_and_fit_model(False, step_number = '1')

#model.save('model_step_1.hdf5')

model.load_weights('best_model.hdf5')

In [ ]:
def calc_scores():
    return model.evaluate_generator(valid_generator, steps = len(valid_generator), verbose = 1)

scores = calc_scores()
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
def draw_fig():
    if history == None:
        return
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_fig = range(len(acc))

    plt.plot(epochs_fig, acc, 'g', label = 'Training acc')
    plt.plot(epochs_fig, val_acc, 'r', label = 'Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.figure()

    plt.plot(epochs_fig, loss, 'g', label = 'Training loss')
    plt.plot(epochs_fig, val_loss, 'r', label = 'Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()
    
draw_fig()

## Этап 2

In [ ]:
EPOCHS       = 10
BATCH_SIZE   = 16
LR           = 1e-4
END_LR       = 1e-5
DECAY_STEPS  = 100000

In [ ]:
base_model.trainable = True
# Замораживаем половину базовой модели
fine_tune_at = len(base_model.layers) // 2
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

In [ ]:
history, model = build_and_fit_model(False, step_number = '2')

#model.save('model_step_2.hdf5')

model.load_weights('best_model.hdf5')

In [ ]:
scores = calc_scores()
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
draw_fig()

## Этап 3

In [ ]:
EPOCHS       = 12
BATCH_SIZE   = 10
LR           = 1e-5
END_LR       = 1e-6
DECAY_STEPS  = 100000

In [ ]:
# Размораживаем всю базовую модель
for layer in base_model.layers:
    #if not isinstance(layer, BatchNormalization): 
    layer.trainable = True

In [ ]:
history, model = build_and_fit_model(False, step_number = '3')

#model.save('model_step_3.hdf5')

model.load_weights('best_model.hdf5')

In [ ]:
scores = calc_scores()
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
draw_fig()

In [ ]:
!rm -d -r '/dev/shm/'

In [ ]:
def model_cnn():
    model = Sequential()

    # Add convolutional layer consisting of 32 filters and shape of 3x3 with ReLU activation
    # We want to preserve more information for following layers so we use padding
    # 'Same' padding tries to pad evenly left and right, 
    # but if the amount of columns to be added is odd, it will add the extra column to the right
    model.add(Conv2D(280, kernel_size = (3,3), activation='relu', input_shape = input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(280, kernel_size = (3,3), activation='relu'))
    model.add(BatchNormalization())

    # Add convolutional layer consisting of 32 filters and shape of 5x5 with ReLU activation
    # We give strides=2 for space between each sample on the pixel grid
    model.add(Conv2D(280, kernel_size = (5,5), strides=2, padding='same', activation='relu'))
    model.add(BatchNormalization())
    # Dropping %40 of neurons
    model.add(Dropout(0.4))
    
    model.add(Conv2D(64, kernel_size = (3,3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (3,3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (5,5), strides=2, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(128, kernel_size = 4, activation='relu'))
    model.add(BatchNormalization())
    # To be able to merge into fully connected layer we have to flatten
    model.add(Flatten())
    model.add(Dropout(0.4))
    
    # Lets add softmax activated neurons as much as number of classes
    model.add(Dense(CLASS_NUM, activation = "softmax"))
    
    # Compile the model with loss and metrics
    model.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(lr = learning_rate_fn, amsgrad = True), metrics = ["accuracy"])       
    
    return model

models = []
#models.append(model_cnn())

# Submission

In [ ]:
#!unzip '../input/sf-dl-car-classification/test.zip' -d '../working/car/'
print('Распаковка картинок')
with zipfile.ZipFile(INPUT_PATH + 'test.zip',"r") as z:
    z.extractall(PICTURE_SPLIT_PATH)
print('Распаковка завершена')

In [ ]:
submission_df = pd.read_csv(INPUT_PATH + 'sample-submission.csv')

In [ ]:
def save_sub(pred, train_generator, test_generator):
    predictions = np.argmax(pred, axis = -1)
    label_map = (train_generator.class_indices)
    label_map = dict((v,k) for k,v in label_map.items())
    predictions = [label_map[k] for k in predictions]
    
    filenames_with_dir = test_generator.filenames
    submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns = ['Id', 'Category'])
    submission['Id'] = submission['Id'].replace('test_upload/','')
    submission.to_csv('submission_TTA.csv', index = False)
    submission.head()
    print('Save submit')

In [ ]:
def sub():
    test_generator = datagen.flow_from_dataframe(
        dataframe = submission_df,
        directory = PICTURE_SPLIT_PATH + 'test_upload/',
        x_col = 'Id',
        y_col = None,
        shuffle = False,
        class_mode = None,
        seed = RANDOM_SEED,
        target_size = (IMG_SIZE, IMG_SIZE),
        batch_size = BATCH_SIZE)
    
    test_generator.reset()
    predictions = model.predict_generator(test_generator, steps=len(test_generator), verbose=1) 
    
    save_sub(predictions, train_generator, test_generator)

In [ ]:
def sub_tta():        
    test_generator = datagen.flow_from_dataframe( 
        dataframe = submission_df,
        directory = PICTURE_SPLIT_PATH + 'test_upload/',
        x_col = "Id",
        y_col = None,
        shuffle = False,
        class_mode = None,
        seed = RANDOM_SEED,
        target_size = (IMG_SIZE, IMG_SIZE),
        batch_size = BATCH_SIZE)
            
    tta_steps = 10
    predictions = []
    models = []

    for i in range(tta_steps):
        #models[i].predict(test_generator, steps = len(test_generator), verbose = 1) 
        preds = model.predict(test_generator, steps = len(test_generator), verbose = 1) 
        predictions.append(preds)

    predictions = np.mean(predictions, axis = 0)
    
    save_sub(predictions, train_generator, test_generator)

In [ ]:
sub_tta()

In [ ]:
files = os.listdir("/kaggle/working")

for filename in files:
    print(filename)